# Загрузка данных анкет и проведение МАИ, анализ полученных результатов
Данные: заполненные анкеты экспертов (таблицы Excel) 

In [1]:
import re
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
from random import randint

In [2]:
df = pd.read_excel(r'C:\Users\user\предпосылки\Экс13.xlsx')
df.head()

,Тип нарушения,Документация,Экспертиза промышленной безопасности,Готовность к ликвидации последствий аварий и инцидентов,Оформление информационными знаками и знаками безопасности,Содержание объектов и оборудования,Охранные зоны (ОЗ),Организация рабочего процесса и безопасности персонала
0,Документация,1.0,1.0,5.000000,1.0,2.0,2.0,3
1,Экспертиза промышленной безопасности,1.0,1.0,2.000000,5.0,1.0,6.0,3
2,Готовность к ликвидации последствий аварий и и...,0.2,0.5,1.000000,7.0,2.0,6.0,4
3,Оформление информационными знаками и знаками б...,1.0,0.2,0.142857,1.0,2.0,2.0,1
4,Содержание объектов и оборудования,0.5,1.0,0.500000,0.5,1.0,6.0,1


In [3]:
print('Число строк и столбцов в наборе данных:\n', df.shape)

Число строк и столбцов в наборе данных:
 (7, 8)


In [4]:
# типы столбцов 
df.dtypes

Тип нарушения                                                 object
Документация                                                 float64
Экспертиза промышленной безопасности                         float64
Готовность к ликвидации последствий аварий и инцидентов      float64
Оформление информационными знаками и знаками безопасности    float64
Содержание объектов и оборудования                           float64
Охранные зоны (ОЗ)                                           float64
Организация рабочего процесса и безопасности персонала         int64
dtype: object

In [5]:
#удаление столбца типа object для дальнейшей работы с фреймом 
df.pop('Тип нарушения') 
print(df)

   Документация   Экспертиза промышленной безопасности  \
0       1.000000                              1.000000   
1       1.000000                              1.000000   
2       0.200000                              0.500000   
3       1.000000                              0.200000   
4       0.500000                              1.000000   
5       0.500000                              0.166667   
6       0.333333                              0.333333   

   Готовность к ликвидации последствий аварий и инцидентов  \
0                                           5.000000         
1                                           2.000000         
2                                           1.000000         
3                                           0.142857         
4                                           0.500000         
5                                           0.166667         
6                                           0.250000         

   Оформление информационными знаками 

In [6]:
# типы столбцов 
df.dtypes

Документация                                                 float64
Экспертиза промышленной безопасности                         float64
Готовность к ликвидации последствий аварий и инцидентов      float64
Оформление информационными знаками и знаками безопасности    float64
Содержание объектов и оборудования                           float64
Охранные зоны (ОЗ)                                           float64
Организация рабочего процесса и безопасности персонала         int64
dtype: object

In [7]:
# также меням тип данных Документации
df['Документация '] = df['Документация '].astype(float)

In [8]:
df.loc[:, df.columns[df.dtypes == 'float64']].head(7)

,Документация,Экспертиза промышленной безопасности,Готовность к ликвидации последствий аварий и инцидентов,Оформление информационными знаками и знаками безопасности,Содержание объектов и оборудования,Охранные зоны (ОЗ)
0,1.000000,1.000000,5.000000,1.0,2.000000,2.000000
1,1.000000,1.000000,2.000000,5.0,1.000000,6.000000
2,0.200000,0.500000,1.000000,7.0,2.000000,6.000000
3,1.000000,0.200000,0.142857,1.0,2.000000,2.000000
4,0.500000,1.000000,0.500000,0.5,1.000000,6.000000
5,0.500000,0.166667,0.166667,0.5,0.166667,1.000000
6,0.333333,0.333333,0.250000,1.0,1.000000,0.333333


In [9]:
DF_1=df[:].copy()

In [10]:
#Найдем сумму всех столбцов, для того чтобы в дальнейшем пересчитать полученные значения
df.sum ()

Документация                                                  4.533333
Экспертиза промышленной безопасности                          4.200000
Готовность к ликвидации последствий аварий и инцидентов       9.059524
Оформление информационными знаками и знаками безопасности    16.000000
Содержание объектов и оборудования                            9.166667
Охранные зоны (ОЗ)                                           23.333333
Организация рабочего процесса и безопасности персонала       16.000000
dtype: float64

In [11]:
# Считаем значения для нашей матрицы, путем деления каждого значения в столбце на сумму столбца
df['new_Документация'] = df['Документация '].astype(float).div(df['Документация '].sum())
df['new_Экспертиза'] = df['Экспертиза промышленной безопасности'].astype(float).div(df['Экспертиза промышленной безопасности'].sum())
df['new_Готовность'] = df['Готовность к ликвидации последствий аварий и инцидентов'].astype(float).div(df['Готовность к ликвидации последствий аварий и инцидентов'].sum())
df['new_Оформление'] = df['Оформление информационными знаками и знаками безопасности'].astype(float).div(df['Оформление информационными знаками и знаками безопасности'].sum())
df['new_Содержание'] = df['Содержание объектов и оборудования '].astype(float).div(df['Содержание объектов и оборудования '].sum())
df['new_ОЗ'] = df['Охранные зоны (ОЗ)'].astype(float).div(df['Охранные зоны (ОЗ)'].sum())
df['new_Организация'] = df['Организация рабочего процесса и безопасности персонала'].astype(float).div(df['Организация рабочего процесса и безопасности персонала'].sum())

In [12]:
df.head()

,Документация,Экспертиза промышленной безопасности,Готовность к ликвидации последствий аварий и инцидентов,Оформление информационными знаками и знаками безопасности,Содержание объектов и оборудования,Охранные зоны (ОЗ),Организация рабочего процесса и безопасности персонала,new_Документация,new_Экспертиза,new_Готовность,new_Оформление,new_Содержание,new_ОЗ,new_Организация
0,1.0,1.0,5.000000,1.0,2.0,2.0,3,0.220588,0.238095,0.551905,0.06250,0.218182,0.085714,0.1875
1,1.0,1.0,2.000000,5.0,1.0,6.0,3,0.220588,0.238095,0.220762,0.31250,0.109091,0.257143,0.1875
2,0.2,0.5,1.000000,7.0,2.0,6.0,4,0.044118,0.119048,0.110381,0.43750,0.218182,0.257143,0.2500
3,1.0,0.2,0.142857,1.0,2.0,2.0,1,0.220588,0.047619,0.015769,0.06250,0.218182,0.085714,0.0625
4,0.5,1.0,0.500000,0.5,1.0,6.0,1,0.110294,0.238095,0.055191,0.03125,0.109091,0.257143,0.0625


In [13]:
# Получили новую таблицу теперь отделим значения нашей матрицы от основной таблицы
df_new = df[['new_Документация', 'new_Экспертиза', 'new_Готовность', 'new_Оформление', 'new_Содержание', 'new_ОЗ', 'new_Организация']]
df_new

,new_Документация,new_Экспертиза,new_Готовность,new_Оформление,new_Содержание,new_ОЗ,new_Организация
0,0.220588,0.238095,0.551905,0.06250,0.218182,0.085714,0.1875
1,0.220588,0.238095,0.220762,0.31250,0.109091,0.257143,0.1875
2,0.044118,0.119048,0.110381,0.43750,0.218182,0.257143,0.2500
3,0.220588,0.047619,0.015769,0.06250,0.218182,0.085714,0.0625
4,0.110294,0.238095,0.055191,0.03125,0.109091,0.257143,0.0625
5,0.110294,0.039683,0.018397,0.03125,0.018182,0.042857,0.1875
6,0.073529,0.079365,0.027595,0.06250,0.109091,0.014286,0.0625


In [14]:
#Вычислим средние значение по строке и запишем полученный результот в mean
df_new['mean'] = df_new.mean(axis=1)
df_new.head()

C:\Users\user\anaconda3\envs\envpy3.5\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,new_Документация,new_Экспертиза,new_Готовность,new_Оформление,new_Содержание,new_ОЗ,new_Организация,mean
0,0.220588,0.238095,0.551905,0.06250,0.218182,0.085714,0.1875,0.223498
1,0.220588,0.238095,0.220762,0.31250,0.109091,0.257143,0.1875,0.220811
2,0.044118,0.119048,0.110381,0.43750,0.218182,0.257143,0.2500,0.205196
3,0.220588,0.047619,0.015769,0.06250,0.218182,0.085714,0.0625,0.101839
4,0.110294,0.238095,0.055191,0.03125,0.109091,0.257143,0.0625,0.123366


In [15]:
#проверка, среднее значение в сумме должно быть равно 1
df_new['mean'].sum()

1.0

In [16]:
# Далее проведем проверку на согласованность, 
#для этого создадим из нашей полученной выше таблицы матрицы для дальнейшего их перемножения
df_new8 =df_new['mean']

# удаляем из нашей таблицы значениея столбцы mean
#df_new.pop('mean')

df_new1 =DF_1.loc[0]

df_new2 =DF_1.loc[1]

df_new3 =DF_1.loc[2]

df_new4 =DF_1.loc[3]

df_new5 =DF_1.loc[4]

df_new6 =DF_1.loc[5]

df_new7 =DF_1.loc[6]

In [17]:
#Проверка на согласованность
res1 = np.dot(df_new1,df_new8)
print(res1)
res2 = np.dot(df_new2,df_new8)
print(res2)
res3 = np.dot(df_new3,df_new8)
print(res3)
res4 = np.dot(df_new4,df_new8)
print(res4)
res5 = np.dot(df_new5,df_new8)
print(res5)
res6 = np.dot(df_new6,df_new8)
print(res6)
res7 = np.dot(df_new7,df_new8)
print(res7)

2.1307061383982795
2.055200651299051
1.9491114485318448
0.8348582072687837
1.0548497536977353
0.502053690692746
0.5072148359537394


In [18]:
sum_new = res1 + res2 + res3 + res4 + res5 + res6 + res7
print(sum_new)

9.033994725842179


In [19]:
# Вычислем значения CI, RI, CR
CI = (sum_new - 7)/(7-1)
print(CI)
RI = (1.98*(7-2))/7
print(RI)
CR = CI/RI
print(CR)

0.3389991209736965
1.4142857142857144
0.23969634816321975


In [20]:
# Получилось, что значение CR=0.24, что большее 0,1 значит матрицу нельзя считать согласованной
# Проделаем аналогичные действия с другими экспертами